In [62]:
import logging
import pypsa
import os.path
import numpy as np  
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
# running the jupyter notebook on the compute nodes doesnt build the path as expected, you have to manually do this
import IPython

working_directory = os.path.dirname(IPython.extract_module_locals()[1]['__vsc_ipynb_file__'])
workflow_dir = os.path.dirname(working_directory)
scripts_dir = os.path.join(workflow_dir, "scripts")
root_dir = os.path.dirname(workflow_dir)

os.chdir(scripts_dir)

# from make_summary import assign_carriers
from _helpers import configure_logging, mock_snakemake
from constants import PLOT_COST_UNITS, PLOT_CAP_UNITS,PLOT_SUPPLY_UNITS

logger = logging.getLogger(__name__)
PLANNING_YEAR = 2055
snakemake = mock_snakemake(
    "plot_network",
    snakefile_path=os.path.abspath("../"),
    topology="current+FCG",
    co2_pathway="exp175default",
    planning_horizons=PLANNING_YEAR,
    heating_demand="positive",
)

configure_logging(snakemake, logger=logger)
config = snakemake.config
tech_colors = config["plotting"]["tech_colors"]


ntw_path = snakemake.input.network
# ntw_path = f"/home/ivanra/downloads/PaperResultsXiaowei_networks/postnetwork-ll-current+Neighbor-exponential175-{PLANNING_YEAR}.nc"
ntw_path = "/p/tmp/yanleizh/PyPSA-China-PIK/results/v-0.2.0_test_co2_pr_baseline_0411_NM_lowCB/overnight_co2pw-exp175default_topo-current+FCG_proj-positive/postnetworks/ntwk_2060.nc"
n = pypsa.Network(ntw_path)
results_dir = os.path.dirname(os.path.dirname(ntw_path))
results_dir 


2025-04-11 15:03:33,190 - _helpers.py - INFO - =========== NEW RUN ===========
2025-04-11 15:03:33,190 - _helpers.py - INFO - =========== NEW RUN ===========
2025-04-11 15:03:33,190 - _helpers.py - INFO - =========== NEW RUN ===========
2025-04-11 15:03:33,190 - _helpers.py - INFO - =========== NEW RUN ===========
2025-04-11 15:03:33,190 - _helpers.py - INFO - =========== NEW RUN ===========
2025-04-11 15:03:33,190 - _helpers.py - INFO - =========== NEW RUN ===========
2025-04-11 15:03:33,190 - _helpers.py - INFO - =========== NEW RUN ===========
2025-04-11 15:03:33,190 - _helpers.py - INFO - =========== NEW RUN ===========
2025-04-11 15:03:33,190 - _helpers.py - INFO - =========== NEW RUN ===========
INFO:__main__:=========== NEW RUN ===========
INFO:pypsa.io:Imported network ntwk_2060.nc has buses, carriers, generators, global_constraints, links, loads, storage_units, stores


'/p/tmp/yanleizh/PyPSA-China-PIK/results/v-0.2.0_test_co2_pr_baseline_0411_NM_lowCB/overnight_co2pw-exp175default_topo-current+FCG_proj-positive'

### Path setting

In [63]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

# 新建 PDF 文件（只运行一次）
pdf_path = "NM_lowCB_budget_mediumcost.pdf"
pdf_pages = PdfPages(pdf_path)
STANDARD_FIGSIZE = (12, 7)  # 或 A4 landscape (11.69, 8.27)
def auto_sized_subplot_grid(n_panels, ncols=6, height_per_row=6, width=24):
    nrows = (n_panels + ncols - 1) // ncols
    figsize = (width, nrows * height_per_row)
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    return fig, axes.flatten()


In [64]:
n.global_constraints

,type,investment_period,carrier_attribute,sense,constant,mu
GlobalConstraint,,,,,,
co2_limit,primary_energy,NaN,co2_emissions,<=,2.983348e+08,-5.216479e-14


In [65]:
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages
import random

# --- Data Preparation ---
capacity = n.statistics.optimal_capacity(groupby=["carrier", "location"]) / 1e3  # GW
generation = n.statistics.supply(groupby=["carrier", "location"]) / 1e6  # TWh

df_cap = capacity.unstack("carrier").fillna(0)
df_gen = generation.unstack("carrier").fillna(0)

df_cap_generator = df_cap.loc["Generator"]
df_gen_generator = df_gen.loc["Generator"]

df_cap_grouped = df_cap_generator.groupby("location").sum()
df_gen_grouped = df_gen_generator.groupby("location").sum()

# 获取在容量和发电量数据中都存在的carriers
common_carriers = list(set(df_cap_grouped.columns) & set(df_gen_grouped.columns))

# 修改这部分代码，为没有颜色的技术设置随机颜色
tech_colors = config["plotting"]["tech_colors"]

# 为没有预定义颜色的技术生成随机颜色
for carrier in common_carriers:
    if carrier not in tech_colors:
        random_color = '#%02X%02X%02X' % (
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255)
        )
        tech_colors[carrier] = random_color

# 使用更新后的颜色映射
colors = [tech_colors[c] for c in common_carriers]

# 只使用共同的carriers
df_cap_plot = df_cap_grouped[common_carriers]
df_gen_plot = df_gen_grouped[common_carriers]

# --- Plotting ---
fig, axes = plt.subplots(2, 1, figsize=(11.69, 10), sharex=True, gridspec_kw={"hspace": 0.3})

# Main title
fig.suptitle("Optimal Capacity and Generation by Technology", fontsize=16, y=0.98)

# Top plot: Installed capacity
df_cap_plot.plot(kind="bar", stacked=True, ax=axes[0], color=colors)
axes[0].set_title("Optimal Capacity (GW)", fontsize=13)
axes[0].set_ylabel("GW")
axes[0].legend(title="Technology", bbox_to_anchor=(1.01, 1), loc="upper left", fontsize=10, title_fontsize=11)

# Bottom plot: Generation
df_gen_plot.plot(kind="bar", stacked=True, ax=axes[1], color=colors)
axes[1].set_title("Annual Generation (TWh)", fontsize=13)
axes[1].set_ylabel("TWh")
axes[1].set_xticklabels(df_cap_plot.index, rotation=45, ha="right")
axes[1].legend().remove()

# Layout adjustments
plt.subplots_adjust(top=0.90, bottom=0.1)
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Save to PDF
pdf_pages.savefig(fig)
plt.close(fig)

/p/tmp/yanleizh/anaconda/ipykernel_2484537/2255581575.py:63: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 1, 0.95])


In [66]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# --- 统计数据 ---
capex_ = n.statistics.expanded_capex().sum()
installed_capex_ = n.statistics.installed_capex().sum()
opex_ = n.statistics.opex().sum()
em_price_gas = float(n.statistics.supply(comps="Generator")["gas"]*n.global_constraints.at["co2_limit", "mu"]*0.2/-0.43)
print(f"{em_price_gas:.2e}")
costs_ = pd.DataFrame({
    "OPEX": opex_,
    "CAPEX": capex_,
    "CO2_cost": em_price_gas,
    "total": capex_ + installed_capex_ + opex_ + em_price_gas,
}, index=["Costs"]).stack()

revenue_ = n.statistics.revenue(comps="Load", bus_carrier="AC").sum()
costs_.loc[("Revenue", "AC")] = -revenue_

# --- 组合成 DataFrame ---
all_ = pd.concat({"2060 nobudget": costs_}, names=["CO2 control"])
all_.index.names = ["CO2 control", "Cat", "Type"]
all_ = pd.DataFrame(all_, columns=["Value"]).reset_index()

# --- 画图 ---
g = sns.catplot(data=all_, x="Cat", y="Value", hue="Type", col="CO2 control",
                kind="bar", dodge=True, alpha=0.8)

# 添加图标题（可选）
g.fig.suptitle("Total cost and revenue", fontsize=14, y=1.05)

# 保存到 PDF（你之前初始化的 pdf_pages）
pdf_pages.savefig(g.fig)
plt.close(g.fig)
n.global_constraints.at["co2_limit", "mu"]
gcon = n.global_constraints

columns_to_check = ["constant", "mu", "value"]
existing_columns = [col for col in columns_to_check if col in gcon.columns]

if "co2_limit" in gcon.index and existing_columns:
    print(gcon.loc["co2_limit", existing_columns])
else:
    print("⚠️ CO2 constraint or expected columns not found.")



3.29e-06
constant    298334764.093623
mu                      -0.0
Name: co2_limit, dtype: object


In [67]:
# 指定分组维度
groupby = ["carrier", "location"]

# 获取三项数据
capex = n.statistics.expanded_capex(groupby=groupby)
opex = n.statistics.opex(groupby=groupby)
revenue = n.statistics.revenue(groupby=groupby)

# 获取完整索引：三个 Series 的 index 的并集
full_index = capex.index.union(opex.index).union(revenue.index)

# 对齐所有数据到完整索引，并填补 NaN 为 0
capex = capex.reindex(full_index, fill_value=0)
opex = opex.reindex(full_index, fill_value=0)
revenue = revenue.reindex(full_index, fill_value=0)

# 计算 total cost 和 profit
total_cost = capex + opex
print(total_cost)
profit = revenue - total_cost

# 汇总成 DataFrame
comparison = pd.DataFrame({
    "Revenue (€/year)": revenue,
    "CAPEX (€/year)": capex,
    "OPEX (€/year)": opex,
    "Total Cost (€/year)": total_cost,
    "Profit (€/year)": profit,
})

comparison = comparison[comparison.index.get_level_values("component") != "Load"]
# 可视化：按地区分子图
import matplotlib.pyplot as plt

# 获取所有地区
locations = comparison.index.get_level_values("location").unique()
ncols = 5
nrows = (len(locations) + ncols - 1) // ncols

fig, axes = auto_sized_subplot_grid(len(locations), ncols=6)
axes = axes.flatten()

for i, loc in enumerate(locations):
    ax = axes[i]
    df_loc = comparison.xs(loc, level="location")

    # 如果该地区为空则跳过
    if df_loc.empty:
        ax.set_visible(False)
        continue

    # 排序（这里可按 revenue 或 profit）
    df_loc = df_loc.sort_values("Revenue (€/year)", ascending=False)

    # 绘图
    df_loc[["Revenue (€/year)", "Total Cost (€/year)"]].plot(
        kind="bar", ax=ax, width=0.8
    )

    ax.set_title(f"{loc}")
    ax.set_xlabel("")
    ax.set_ylabel("€/year")
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis="x", labelrotation=45)
    # 改进部分 ✅
    ax.tick_params(axis="x", labelrotation=60)
    for label in ax.get_xticklabels():
        label.set_ha("right")
# 移除多余子图
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# 添加统一图例
handles, labels = axes[0].get_legend_handles_labels()
# fig.legend(handles, labels, loc="RIGHTER", bbox_to_anchor=(0.5, 1.02), ncol=1)
fig.suptitle("Revenue and Cost by Region", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.99])  # 留出 suptitle 空间
pdf_pages.savefig(fig)
plt.close(fig)



component    carrier             location     
Generator    coal                InnerMongolia    5.297432e+09
             gas                 InnerMongolia    2.723697e+09
             onwind              InnerMongolia    1.129932e+10
             solar               InnerMongolia    3.683888e+09
Link         H2 Electrolysis     InnerMongolia    2.000000e-04
             H2 fuel cell        InnerMongolia    3.000000e-04
             Sabatier            InnerMongolia    2.300000e-04
             battery             InnerMongolia    2.554249e+08
             battery discharger  InnerMongolia    0.000000e+00
             gas OCGT            InnerMongolia    2.489063e+09
Load                             InnerMongolia    0.000000e+00
StorageUnit  PHS                 InnerMongolia    0.000000e+00
Store        H2                  InnerMongolia    2.800000e-04
             battery             InnerMongolia    1.243168e+09
dtype: float64


In [68]:
from _pypsa_helpers import calc_lcoe

# 获取 LCOE 和 MV 数据
df = calc_lcoe(n, groupby=["carrier", "location"])
# 获取所有地区并排序（按平均 MV 排序）
mean_mv = df["MV"].groupby("location").mean()
locations = mean_mv.sort_values(ascending=True).index

# 图布局参数
ncols = 5
nrows = (len(locations) + ncols - 1) // ncols
fig, axes = auto_sized_subplot_grid(len(locations), ncols=6)
axes = axes.flatten()

for i, loc in enumerate(locations):
    ax = axes[i]
    df_loc = df.xs(loc, level="location")
    
    if df_loc.empty:
        ax.set_visible(False)
        continue

    df_loc[["MV", "LCOE"]].plot(kind="bar", ax=ax, width=0.8)
    ax.set_title(loc)
    ax.set_xlabel("")
    ax.set_ylabel("€/MWh")
    ax.grid(True, alpha=0.3)

    # 改进部分 ✅
    ax.tick_params(axis="x", labelrotation=60)
    for label in ax.get_xticklabels():
        label.set_ha("right")


# 删除多余子图
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])
fig.suptitle("MV and LCOE by Region", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.99])  # 留出 suptitle 空间
pdf_pages.savefig(fig)
plt.close(fig)


/p/tmp/yanleizh/PyPSA-China-PIK/workflow/scripts/_pypsa_helpers.py:108: RuntimeWarning: divide by zero encountered in scalar divide
  profits["LCOE"] = profits.apply(lambda row: (row.CAPEX + row.OPEX)/row.supply, axis=1)
/p/tmp/yanleizh/PyPSA-China-PIK/workflow/scripts/_pypsa_helpers.py:108: RuntimeWarning: invalid value encountered in scalar divide
  profits["LCOE"] = profits.apply(lambda row: (row.CAPEX + row.OPEX)/row.supply, axis=1)
/p/tmp/yanleizh/PyPSA-China-PIK/workflow/scripts/_pypsa_helpers.py:110: RuntimeWarning: divide by zero encountered in scalar divide
  lambda row: (row.CAPEX_wBROWN + row.OPEX)/row.supply, axis=1)
/p/tmp/yanleizh/PyPSA-China-PIK/workflow/scripts/_pypsa_helpers.py:110: RuntimeWarning: invalid value encountered in scalar divide
  lambda row: (row.CAPEX_wBROWN + row.OPEX)/row.supply, axis=1)
/p/tmp/yanleizh/PyPSA-China-PIK/workflow/scripts/_pypsa_helpers.py:111: RuntimeWarning: divide by zero encountered in scalar divide
  profits["MV"] = profits.apply(lamb

In [69]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# --- 实际容量 ---
df_opt = n.statistics.optimal_capacity(comps="Generator", groupby=["location", "carrier"]) / 1e3
df_opt = df_opt.unstack().fillna(0)
df_opt[df_opt < 1e-3] = 0

# --- 最大容量 ---
gen = n.generators.copy()
gen = gen[gen.p_nom_extendable]
df_max = gen.groupby(["location", "carrier"])["p_nom_max"].sum().unstack().fillna(0) / 1e3

# 记录哪些位置是 inf
mask_inf = gen.groupby(["location", "carrier"])["p_nom_max"].max().unstack()
mask_inf = mask_inf == np.inf

# 在 df_max 和 df_opt 中都移除这些 inf 技术
for loc in mask_inf.index:
    if loc in df_max.index:
        drop_carriers = mask_inf.columns[mask_inf.loc[loc]]
        df_max.loc[loc, drop_carriers] = np.nan
        df_opt.loc[loc, drop_carriers] = np.nan

df_max = df_max.fillna(0)
df_opt = df_opt.fillna(0)

# --- 对齐 ---
all_locations = sorted(set(df_opt.index) | set(df_max.index))
all_carriers = sorted(set(df_opt.columns) | set(df_max.columns))

df_opt = df_opt.reindex(index=all_locations, columns=all_carriers).fillna(0)
df_max = df_max.reindex(index=all_locations, columns=all_carriers).fillna(0)

# --- 排序 ---
locations = df_opt.sum(axis=1).sort_values().index

# --- 绘图 ---
ncols = 5
nrows = (len(locations) + ncols - 1) // ncols
fig, axes = auto_sized_subplot_grid(len(locations), ncols=6)
axes = axes.flatten()

for i, loc in enumerate(locations):
    ax = axes[i]
    y_max = df_max.loc[loc]
    y_opt = df_opt.loc[loc]

    # 只保留该地区“有限”的技术
    mask = (y_max > 0) | (y_opt > 0)
    y_max = y_max[mask]
    y_opt = y_opt[mask]
    carriers = y_max.index.tolist()

    if len(carriers) == 0:
        ax.set_visible(False)
        continue

    x = range(len(carriers))
    width = 0.35

    ax.bar([xi - width/2 for xi in x], y_max.values, width=width, color="lightgray", label="Potential")
    ax.bar([xi + width/2 for xi in x], y_opt.values, width=width, color="steelblue", label="Capacity")

    ax.set_title(loc)
    ax.set_xticks(x)
    ax.set_xticklabels(carriers, rotation=45, ha="right")
    ax.set_ylabel("Capacity (GW)")
    ax.grid(True, axis="y", alpha=0.3)

    if i == 0:
        ax.legend()

# 删除多余子图
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])
fig.suptitle("Max and opt", fontsize=14)
plt.tight_layout(rect=[0, 0, 1, 0.99])  # 留出 suptitle 空间
pdf_pages.savefig(fig)
plt.close(fig)


In [70]:
pdf_pages.close()
print("✅ 所有图已写入 PDF 文件！")


✅ 所有图已写入 PDF 文件！
